In [11]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

# global varuables
w = h = 9

In [2]:
#
# Reads band images and index images
#
def read_img(site, _s):
    data = []

    for i in range(1, 8):
        if _s == 'B':
            filename = "input/"+ site +"/B" + str(i) + "_250_B.tif"
        else:
            filename = "input/"+ site +"/B" + str(i) + "_250.tif"
            
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        data.append(img)

    index = ["NDVI.tif", "NDWI.tif", "NDBI.tif"]
    for i in index:
        filename = "input/"+ site +"/" + i
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        data.append(img)

#     cv2.imshow("Image", data[9])
#     cv2.waitKey(0)
    
    return data

i_img = read_img("Landsat8-A", 'A')
test_img = read_img("Landsat8-B", 'B')

In [3]:
# Training

def crop_train(_img, _c, site):
    _class = ""
    if(_c == 0):
        _class = 'Vegetation'
    elif(_c == 1):
        _class = 'Built-up'
    else:
        _class = 'Water'
        
    _pts = pd.read_excel('input/'+ site +'.xlsx', sheet_name = _class)

    # iterate over rows with iterrows()
    for index, row in _pts.iterrows():
        x = row['X'] - 4
        y = row['Y'] - 4
        
        for img in _img:        
            crop_img = img[y:y+h, x:x+w]
            med = np.median(crop_img) # get median of 9x9 image
            X_tr_list.append(med)
        y_tr.append(_c)


In [4]:
# Testing

def crop_test(_img):
    _pts = pd.read_excel('input/site-b.xlsx')

    for index, row in _pts.iterrows():
        x = row['X'] - 4
        y = row['Y'] - 4
        
        if(row['Class'] == 255):
            _c = 0
        elif(row['Class'] == 128):
            _c = 1
        else:
            _c = 2
        
        for img in _img:        
            crop_img = img[y:y+h, x:x+w]
            med = np.median(crop_img) # get median of 9x9 image
            X_te_list.append(med)
        y_te.append(_c)
        

### Training and Testing Dataset

In [5]:
# TRAINING SET SITE A ------------------------------------------------------------------
X_tr_list = []
y_tr = []

crop_train(i_img, 0, 'site-a')
crop_train(i_img, 1, 'site-a')
crop_train(i_img, 2, 'site-a')

print("Training...")
print(len(X_tr_list))
print(len(y_tr))


# TRAINING SET SITE B ------------------------------------------------------------------
X_te_list = []
y_te = []

crop_test(i_img)
print("Testing...")
print(len(X_te_list))
print(len(y_te))

Training...
2000
200
Testing...
2000
200


### Reshaping Training and Testing Dataset

In [6]:
# TRAINING SET SITE A ------------------------------------------------------------------
X_tr = []
X_tr = np.array(X_tr_list).reshape(-1, 10)

print(X_tr[0].shape)
print(X_tr[0], y_tr[0])

# TESTING SET SITE B ------------------------------------------------------------------
X_te = []
X_te = np.array(X_te_list).reshape(-1, 10)

print(X_te[0].shape)
print(X_te[0], y_te[0])

(10,)
[ 36.  32.  29.  26.  59.  37.  26. 213.  41.  62.] 0
(10,)
[ 37.  33.  30.  27.  54.  36.  26. 198.  59.  85.] 2


### Define Keras Model

In [7]:
# define the keras model
# The model expects rows of data with 10 variables (the input_dim=10 argument)
# The first hidden layer has 12 nodes and uses the relu activation function.
# The second hidden layer has 8 nodes and uses the relu activation function.
# The output layer has one node and uses the sigmoid activation function.

model = Sequential()
model.add(Dense(9, input_dim=10, activation='relu'))
model.add(Dense(7, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


### Compile Model

In [8]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Fit Model

In [9]:
# fit the keras model on the dataset
model.fit(X_tr, y_tr, epochs=150, batch_size=10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/150
200/200 [==============================] - 0s 1ms/sample - loss: 12.8139 - acc: 0.3050
Epoch 2/150
200/200 [==============================] - 0s 123us/sample - loss: 12.8139 - acc: 0.3050
Epoch 3/150
200/200 [==============================] - 0s 140us/sample - loss: 12.8139 - acc: 0.3050
Epoch 4/150
200/200 [==============================] - 0s 140us/sample - loss: 12.8139 - acc: 0.3050
Epoch 5/150
200/200 [==============================] - 0s 130us/sample - loss: 12.8139 - acc: 0.3050
Epoch 6/150
200/200 [==============================] - 0s 130us/sample - loss: 12.8139 - acc: 0.3050
Epoch 7/150
200/200 [==============================] - 0s 130us/sample - loss: 12.8139 - acc: 0.3050
Epoch 8/150
200/200 [==============================] - 0s 130us/sample - loss: 12.8139 - acc: 0.3050
Epoch 9/150
200/200 [==============================] - 0s 130us/sample - loss: 12.8139 - acc: 0.3050
Epoch 10/150
200/200 [=======================

200/200 [==============================] - 0s 125us/sample - loss: 12.8139 - acc: 0.3050
Epoch 79/150
200/200 [==============================] - 0s 120us/sample - loss: 12.8139 - acc: 0.3050
Epoch 80/150
200/200 [==============================] - 0s 110us/sample - loss: 12.8139 - acc: 0.3050
Epoch 81/150
200/200 [==============================] - 0s 110us/sample - loss: 12.8139 - acc: 0.3050
Epoch 82/150
200/200 [==============================] - 0s 115us/sample - loss: 12.8139 - acc: 0.3050
Epoch 83/150
200/200 [==============================] - 0s 105us/sample - loss: 12.8139 - acc: 0.3050
Epoch 84/150
200/200 [==============================] - 0s 115us/sample - loss: 12.8139 - acc: 0.3050
Epoch 85/150
200/200 [==============================] - 0s 120us/sample - loss: 12.8139 - acc: 0.3050
Epoch 86/150
200/200 [==============================] - 0s 115us/sample - loss: 12.8139 - acc: 0.3050
Epoch 87/150
200/200 [==============================] - 0s 110us/sample - loss: 12.8139 - acc: 

### Evaluate Model

In [10]:
# evaluate the keras model
loss, accuracy = model.evaluate(X_te, y_te)
print('Accuracy: %.2f' % (accuracy*100))

200/200 [==============================] - 0s 265us/sample - loss: 8.7844 - acc: 0.5250
Accuracy: 52.50
